### Compare ppl difference between (fwd vs rev) and (fwd seed2 vs fwd seed3)

In [3]:
import numpy as np
import scipy.stats as stats
from utils import general_utils

In [20]:
def cohens_d_rel(x, y):
    diff_mean = np.mean(x) - np.mean(y)
    diff_var = np.var(x - y, ddof=1)
    return np.abs(diff_mean / np.sqrt(diff_var))

* Validation set

In [25]:
# Bayes fwd and rev models
# (train-sampled and validation sets)
fwd_bayes_val_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd/human_abstracts/all_batches_ppl_validation.npy"
rev_bayes_val_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_rev/human_abstracts/all_batches_ppl_validation.npy"
fwd_bayes_val_seed2_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd_seed2/human_abstracts/all_batches_ppl_validation.npy"
fwd_bayes_val_seed3_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd_seed3/human_abstracts/all_batches_ppl_validation.npy"

fwd_bayes_val_ppls = np.load(fwd_bayes_val_path)
rev_bayes_val_ppls = np.load(rev_bayes_val_path)
fwd_bayes_val_seed2_ppls = np.load(fwd_bayes_val_seed2_path)
fwd_bayes_val_seed3_ppls = np.load(fwd_bayes_val_seed3_path)

In [27]:
# T-test
print(
    "Bayes Val Fwd vs Rev:\n", stats.ttest_rel(fwd_bayes_val_ppls, rev_bayes_val_ppls)
)
print(
    "Bayes Val Fwd vs Fwd seed2:\n",
    stats.ttest_rel(fwd_bayes_val_ppls, fwd_bayes_val_seed2_ppls),
)
print(
    "Bayes Val Fwd vs Fwd seed3:\n",
    stats.ttest_rel(fwd_bayes_val_ppls, fwd_bayes_val_seed3_ppls),
)

Bayes Val Fwd vs Rev:
 TtestResult(statistic=-15.672598361144663, pvalue=1.139724125552024e-54, df=9412)
Bayes Val Fwd vs Fwd seed2:
 TtestResult(statistic=29.117340857422708, pvalue=1.512070129783087e-178, df=9412)
Bayes Val Fwd vs Fwd seed3:
 TtestResult(statistic=44.58779533114187, pvalue=0.0, df=9412)


In [24]:
# Cohen's d
print(
    "Bayes Val Fwd vs Rev:\n",
    cohens_d_rel(fwd_bayes_val_ppls, rev_bayes_val_ppls),
)
print(
    "Bayes Val Fwd vs Fwd seed2:\n",
    cohens_d_rel(fwd_bayes_val_ppls, fwd_bayes_val_seed2_ppls),
)
print(
    "Bayes Val Fwd vs Fwd seed3:\n",
    cohens_d_rel(fwd_bayes_val_ppls, fwd_bayes_val_seed3_ppls),
)

Bayes Val Fwd vs Rev:
 0.16153884553828604
Bayes Val Fwd vs Fwd seed2:
 0.3001149853309472
Bayes Val Fwd vs Fwd seed3:
 0.45957031609684224


* Brainbench differences

In [2]:
def _ppl_original_ppl_altered_and_ppl_diff(PPL_A_and_B, labels, norm=False):
    """
    Args:
        PPL_A_and_B: (N, 2) array of PPL values for A and B
        labels: (N, ) array of labels (0 or 1) where 0 indicates A correct, B incorrect
        norm: normalization method for PPL difference
            False (default): no normalization
            True: normalize by the PPL of the original

    Returns:
        PPL_original: (N, ) array of PPL values of original/correct options
        PPL_altered: (N, ) array of PPL values of altered/incorrect options
        PPL_diff: (N, ) array of PPL difference between altered and original options
    """
    PPL_A = PPL_A_and_B[:, 0]
    PPL_B = PPL_A_and_B[:, 1]
    PPL_original = []
    PPL_altered = []
    PPL_diff = []  # altered - original
    for i, label in enumerate(labels):
        if label == 0:  # A correct, B incorrect
            PPL_original.append(PPL_A[i])
            PPL_altered.append(PPL_B[i])
            diff = PPL_B[i] - PPL_A[i]
            if norm is True:
                diff /= PPL_A[i]
            PPL_diff.append(diff)
        else:
            PPL_original.append(PPL_B[i])
            PPL_altered.append(PPL_A[i])
            diff = PPL_A[i] - PPL_B[i]
            if norm is True:
                diff /= PPL_B[i]
            PPL_diff.append(diff)
    return np.array(PPL_original), np.array(PPL_altered), np.array(PPL_diff)

In [4]:
general_utils.scorer_acc

<function utils.general_utils.scorer_acc(PPL_A_and_B, labels)>

In [6]:
# Bayes fwd and rev models
fwd_bayes_brainbench_ppls_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd/human_abstracts/PPL_A_and_B.npy"
fwd_bayes_brainbench_labels_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd/human_abstracts/labels.npy"
rev_bayes_brainbench_ppls_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_rev/human_abstracts/PPL_A_and_B.npy"
rev_bayes_brainbench_labels_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_rev/human_abstracts/labels.npy"
fwd_bayes_brainbench_ppls_seed2_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd_seed2/human_abstracts/PPL_A_and_B.npy"
fwd_bayes_brainbench_labels_seed2_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd_seed2/human_abstracts/labels.npy"
fwd_bayes_brainbench_ppls_seed3_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd_seed3/human_abstracts/PPL_A_and_B.npy"
fwd_bayes_brainbench_labels_seed3_path = "model_results/gpt2_scratch_neuro_tokenizer_bayes_fwd_seed3/human_abstracts/labels.npy"

fwd_bayes_brainbench_ppls = np.load(fwd_bayes_brainbench_ppls_path)
fwd_bayes_brainbench_labels = np.load(fwd_bayes_brainbench_labels_path)
rev_bayes_brainbench_ppls = np.load(rev_bayes_brainbench_ppls_path)
rev_bayes_brainbench_labels = np.load(rev_bayes_brainbench_labels_path)
fwd_bayes_brainbench_ppls_seed2 = np.load(fwd_bayes_brainbench_ppls_seed2_path)
fwd_bayes_brainbench_labels_seed2 = np.load(fwd_bayes_brainbench_labels_seed2_path)
fwd_bayes_brainbench_ppls_seed3 = np.load(fwd_bayes_brainbench_ppls_seed3_path)
fwd_bayes_brainbench_labels_seed3 = np.load(fwd_bayes_brainbench_labels_seed3_path)

fwd_bayes_brainbench_ppls_original, fwd_bayes_brainbench_ppls_altered, fwd_bayes_brainbench_ppls_diff \
    = _ppl_original_ppl_altered_and_ppl_diff(fwd_bayes_brainbench_ppls, fwd_bayes_brainbench_labels)
rev_bayes_brainbench_ppls_original, rev_bayes_brainbench_ppls_altered, rev_bayes_brainbench_ppls_diff \
    = _ppl_original_ppl_altered_and_ppl_diff(rev_bayes_brainbench_ppls, rev_bayes_brainbench_labels)
fwd_bayes_brainbench_ppls_seed2_original, fwd_bayes_brainbench_ppls_seed2_altered, fwd_bayes_brainbench_ppls_seed2_diff \
    = _ppl_original_ppl_altered_and_ppl_diff(fwd_bayes_brainbench_ppls_seed2, fwd_bayes_brainbench_labels_seed2)
fwd_bayes_brainbench_ppls_seed3_original, fwd_bayes_brainbench_ppls_seed3_altered, fwd_bayes_brainbench_ppls_seed3_diff \
    = _ppl_original_ppl_altered_and_ppl_diff(fwd_bayes_brainbench_ppls_seed3, fwd_bayes_brainbench_labels_seed3)

print(fwd_bayes_brainbench_ppls_original.shape, fwd_bayes_brainbench_ppls_diff.shape)
print(rev_bayes_brainbench_ppls_original.shape, rev_bayes_brainbench_ppls_diff.shape)


(200,) (200,)
(200,) (200,)


In [18]:
# Get acc
fwd_bayes_brainbench_acc = general_utils.scorer_acc(fwd_bayes_brainbench_ppls, fwd_bayes_brainbench_labels)
rev_bayes_brainbench_acc = general_utils.scorer_acc(rev_bayes_brainbench_ppls, rev_bayes_brainbench_labels)
fwd_bayes_brainbench_acc_seed2 = general_utils.scorer_acc(fwd_bayes_brainbench_ppls_seed2, fwd_bayes_brainbench_labels_seed2)
fwd_bayes_brainbench_acc_seed3 = general_utils.scorer_acc(fwd_bayes_brainbench_ppls_seed3, fwd_bayes_brainbench_labels_seed3)
print("Bayes fwd brainbench acc", fwd_bayes_brainbench_acc)
print("Bayes rev brainbench acc", rev_bayes_brainbench_acc)
print("Bayes fwd brainbench acc seed2", fwd_bayes_brainbench_acc_seed2)
print("Bayes fwd brainbench acc seed3", fwd_bayes_brainbench_acc_seed3)

Bayes fwd brainbench acc 0.605
Bayes rev brainbench acc 0.64
Bayes fwd brainbench acc seed2 0.65
Bayes fwd brainbench acc seed3 0.65


In [14]:
# T-test (ppl original)
print(
    "Bayes Brainbench Fwd vs Rev PPL on original:\n",
    stats.ttest_rel(fwd_bayes_brainbench_ppls_original, rev_bayes_brainbench_ppls_original),
)
print(
    "\nBayes Brainbench Fwd vs Fwd seed2 PPL on original:\n",
    stats.ttest_rel(fwd_bayes_brainbench_ppls_original, fwd_bayes_brainbench_ppls_seed2_original),
)
print(
    "\nBayes Brainbench Fwd vs Fwd seed3 PPL on original:\n",
    stats.ttest_rel(fwd_bayes_brainbench_ppls_original, fwd_bayes_brainbench_ppls_seed3_original),
)
print("--" * 50)
# T-test (ppl altered)
print(
    "Bayes Brainbench Fwd vs Rev PPL on altered:\n",
    stats.ttest_rel(fwd_bayes_brainbench_ppls_altered, rev_bayes_brainbench_ppls_altered),
)
print(
    "\nBayes Brainbench Fwd vs Fwd seed2 PPL on altered:\n",
    stats.ttest_rel(fwd_bayes_brainbench_ppls_altered, fwd_bayes_brainbench_ppls_seed2_altered),
)
print(
    "\nBayes Brainbench Fwd vs Fwd seed3 PPL on altered:\n",
    stats.ttest_rel(fwd_bayes_brainbench_ppls_altered, fwd_bayes_brainbench_ppls_seed3_altered),
)

Bayes Brainbench Fwd vs Rev PPL on original:
 TtestResult(statistic=2.117760377137123, pvalue=0.03543670106655037, df=199)

Bayes Brainbench Fwd vs Fwd seed2 PPL on original:
 TtestResult(statistic=0.1079170438457081, pvalue=0.9141702246874246, df=199)

Bayes Brainbench Fwd vs Fwd seed3 PPL on original:
 TtestResult(statistic=2.173954715065984, pvalue=0.030888768384098236, df=199)
----------------------------------------------------------------------------------------------------
Bayes Brainbench Fwd vs Rev PPL on altered:
 TtestResult(statistic=1.9212736465551556, pvalue=0.056126111455141986, df=199)

Bayes Brainbench Fwd vs Fwd seed2 PPL on altered:
 TtestResult(statistic=-0.056193957680839485, pvalue=0.9552436746191996, df=199)

Bayes Brainbench Fwd vs Fwd seed3 PPL on altered:
 TtestResult(statistic=1.9790981254872273, pvalue=0.04918395382984695, df=199)


In [17]:
# Cohens' d (ppl original)
print(
    "Bayes Brainbench Fwd vs Rev Cohen's d",
    cohens_d_rel(fwd_bayes_brainbench_ppls_original, rev_bayes_brainbench_ppls_original),
)
print(
    "Bayes Brainbench Fwd vs Fwd seed2 Cohen's d",
    cohens_d_rel(fwd_bayes_brainbench_ppls_original, fwd_bayes_brainbench_ppls_seed2_original),
)
print(
    "Bayes Brainbench Fwd vs Fwd seed3 Cohen's d",
    cohens_d_rel(fwd_bayes_brainbench_ppls_original, fwd_bayes_brainbench_ppls_seed3_original),
)
print("--" * 50)

# Cohens' d (ppl altered)
print(
    "Bayes Brainbench Fwd vs Rev Cohen's d",
    cohens_d_rel(fwd_bayes_brainbench_ppls_altered, rev_bayes_brainbench_ppls_altered),
)
print(
    "Bayes Brainbench Fwd vs Fwd seed2 Cohen's d",
    cohens_d_rel(fwd_bayes_brainbench_ppls_altered, fwd_bayes_brainbench_ppls_seed2_altered),
)
print(
    "Bayes Brainbench Fwd vs Fwd seed3 Cohen's d",
    cohens_d_rel(fwd_bayes_brainbench_ppls_altered, fwd_bayes_brainbench_ppls_seed3_altered),
)

Bayes Brainbench Fwd vs Rev Cohen's d 0.14974827236018215
Bayes Brainbench Fwd vs Fwd seed2 Cohen's d 0.007630887350887334
Bayes Brainbench Fwd vs Fwd seed3 Cohen's d 0.1537218121015604
----------------------------------------------------------------------------------------------------
Bayes Brainbench Fwd vs Rev Cohen's d 0.1358545623994156
Bayes Brainbench Fwd vs Fwd seed2 Cohen's d 0.003973512853786361
Bayes Brainbench Fwd vs Fwd seed3 Cohen's d 0.13994337051656158


### Thoughts: 
* First, not surprised to see difference in seed2, seed3 as we all know dropout/init makes a difference.
* so the directional asymmetry is as if there is dropout/random init in training. I think it does not mean the asymmetry is not real because differences due to dropout/init are real.
* Perhaps this is interesting in that given fixed dropout/init, only vary direction, induces similar effect whereas the proof shows they have to be strictly equivalent!
* Need to think carefully on combining fwd+rev models as it would be like dropout ensemble yet does not imply fwd+rev is better than fwd+fwd_seed3.
* At the very least this is good sign that the proof somehow holds hence LLMs are not language learning only.